In [3]:
import pandas as pd
import numpy as np
from os import listdir
import re
from models import *

In [4]:
data = pd.read_csv('data/train.csv')
data['as_of_date'] = pd.to_datetime(data.as_of_date)
X,y = data.drop(['Unnamed: 0','entity_id','as_of_date','quickstart_label'],axis=1).applymap(float),data['quickstart_label'].apply(int)

In [5]:
time_frames = pd.read_csv('data/time_windows_frames.csv')

time_frames.sample()

,Unnamed: 0,Start_Train,Start_Threshold,Start_Test,End_Test
3,3,2011-12-01,2012-03-01,2012-04-01,2012-06-01


In [10]:
#loop through time split
for t in range(time_frames.shape[0]):
    date = time_frames.iloc[t].Start_Threshold
    print(date)
    print(t)
    fold_data = data.query(f'as_of_date > "{time_frames.iloc[t].Start_Train}" & as_of_date <= "{time_frames.iloc[t].End_Test}" ' )
    
    X = fold_data.query(f'as_of_date > "{time_frames.iloc[t].Start_Train}" & as_of_date <= "{time_frames.iloc[t].Start_Threshold}" ')\
            .drop(['Unnamed: 0','entity_id','as_of_date','quickstart_label'],axis=1).applymap(float)
    y = fold_data.query(f'as_of_date > "{time_frames.iloc[t].Start_Train}" & as_of_date <= "{time_frames.iloc[t].Start_Threshold}" ')\
            ['quickstart_label'].apply(int)
    X_k_test = fold_data.query(f'as_of_date > "{time_frames.iloc[t].Start_Threshold}" & as_of_date <= "{time_frames.iloc[t].End_Test}" ')\
            .drop(['Unnamed: 0','entity_id','as_of_date','quickstart_label'],axis=1).applymap(float)
    X_k_test_dates = fold_data.query(f'as_of_date > "{time_frames.iloc[t].Start_Threshold}" & as_of_date <= "{time_frames.iloc[t].End_Test}" ')\
            ['as_of_date']
    y_k_test = fold_data.query(f'as_of_date > "{time_frames.iloc[t].Start_Threshold}" & as_of_date <= "{time_frames.iloc[t].End_Test}" ')\
            ['quickstart_label'].apply(int)
    
    #loop throgh models
    for i in range(len(pars)):
            print((t,i))
            ps = list(ParameterSampler(pars[i], n_iter=50, random_state=0))
            if clfs_names[i] == 'RandomForestClassifier':
                [d.update({'n_estimators':int(d['n_estimators'])}) for d in ps]
            
            #loop through model configs
            for j in range(len(ps)):
                print((t,i,j))
                if [t,i,j] not in [list(map(int,re.findall("\d+",l[:10]))) for l in listdir("outputs_models3/")]:
                    print((t,i,j))
                    print(clfs[i])
                    
                    # generate an initialized model with the parameters sampled (ps)
                    gs = clfs[i](**(ps[j]))
                    try:
                        get_p = gs.get_params(False)
                    except:
                        get_p = gs
                    
                    
                    gs = gs.fit(X.values, y.values)
                    y_proba = gs.predict_proba( X_k_test.values)

                    pd.DataFrame(\
                                     {
                                      'date':X_k_test_dates,
                                      'y_proba':np.array(y_proba)[:,1],
                                      'y_true':y_k_test.values}
                                ).to_csv(f'outputs_models3/{t,i,j}_{clfs_names[i]}.csv')
                pd.Series({'Start_Test':time_frames.iloc[t].Start_Test,
                            'get_p':str(get_p)}).to_csv(f'outputs_models3/{t,i,j}_{clfs_names[i]}_date_params.csv')

2011-12-01
0
(0, 0)
(0, 0, 0)
(0, 0, 0)
<class 'sklearn.ensemble._forest.RandomForestClassifier'>


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 287 out of 287 | elapsed:    6.9s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done 287 out of 287 | elapsed:    0.8s finished


(0, 0, 1)
(0, 0, 1)
<class 'sklearn.ensemble._forest.RandomForestClassifier'>


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:   21.5s
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:   35.5s
[Parallel(n_jobs=-1)]: Done 1430 out of 1430 | elapsed:   41.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    2.9s
[Parallel(n_jobs=4)]: Done 1242 tasks      | elapsed:    5.0s
[Parallel(n_jobs=4)]: Done 1430 out of 1430 | elapsed:    5.7s finished


(0, 0, 2)
(0, 0, 2)
<class 'sklearn.ensemble._forest.RandomForestClassifier'>


...

C:\Users\Francisco\AppData\Roaming\Python\Python37\site-packages\sklearn\svm\_base.py:249: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


(9, 6, 48)
(9, 6, 48)
<class 'sklearn.svm._classes.SVC'>
[LibSVM]

C:\Users\Francisco\AppData\Roaming\Python\Python37\site-packages\sklearn\svm\_base.py:249: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


(9, 6, 49)
(9, 6, 49)
<class 'sklearn.svm._classes.SVC'>
[LibSVM]

C:\Users\Francisco\AppData\Roaming\Python\Python37\site-packages\sklearn\svm\_base.py:249: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [ ]:
pd.Series({'Start_Test':time_frames.iloc[t].Start_Test,
'get_p':str(get_p)})

In [ ]:

list(ParameterSampler(param_log_reg, n_iter=2, random_state=0))

In [ ]:

list(ParameterGrid(parameters3))


In [ ]:
SVC(**(list(ParameterGrid(parameters3))[0]))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

logistic = LogisticRegression(solver='saga', tol=1e-2, max_iter=200,
                              random_state=0)
param_log_reg = dict(C=uniform(loc=0, scale=4),
                     penalty=['l2', 'l1'])
clf = RandomizedSearchCV(logistic, distributions, random_state=0)

In [ ]:
from sklearn.model_selection import ParameterSampler
from scipy.stats.distributions import expon
import numpy as np
rng = np.random.RandomState(0)
param_grid = {'a':[1, 2], 'b': expon()}
param_list = list(ParameterSampler(param_grid, n_iter=4, random_state=rng))
rounded_list = [dict((k, round(v, 6)) for (k, v) in d.items())
                 for d in param_list]
rounded_list == [{'b': 0.89856, 'a': 1},
                  {'b': 0.923223, 'a': 1},
                  {'b': 1.878964, 'a': 2},
                  {'b': 1.038159, 'a': 2}]

In [ ]:
param_list

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV

# Setup the parameters and distributions to sample from: param_dist
param_dist = {"max_depth": [3, None],
              "max_features": randint(1, 9),
              "min_samples_leaf": randint(1, 9),
              "criterion": ["gini", "entropy"]}

# Instantiate a Decision Tree classifier: tree
tree = DecisionTreeClassifier()

# Instantiate the RandomizedSearchCV object: tree_cv
tree_cv = RandomizedSearchCV(tree, param_dist, cv=5)

# Fit it to the data
tree_cv.fit(X,y)


In [ ]:
from torch import nn
import torch.nn.functional as F

from skorch import NeuralNetClassifier

class MyModule(nn.Module):
    def __init__(self, num_units=10, nonlin=F.relu):
        super(MyModule, self).__init__()

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(0.5)
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 2)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = F.relu(self.dense1(X))
        X = F.softmax(self.output(X))
        return X


net = NeuralNetClassifier(
    MyModule,
    max_epochs=10,
    lr=0.1,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
)

params = {
    'lr': [0.01, 0.02],
    'max_epochs': [10, 20],
    'module__num_units': [10, 20],
}

gs = GridSearchCV(net, params, refit=False, cv=3, scoring='accuracy')

gs.fit(X, y)
print(gs.best_score_, gs.best_params_)

In [ ]:
import lightgbm as lgb
params = {'learning_rate': np.random.uniform(0, 1),
          'boosting_type': np.random.choice(['gbdt', 'dart', 'goss']),
          'objective': 'regression',
          'metric' :'mae',
          'sub_feature' : np.random.uniform(0, 1),
          'num_leaves' : np.random.randint(20, 300),
          'min_data' : np.random.randint(10, 100),
          'max_depth' : np.random.randint(5, 200)}
#iterations = np.random.randint(10, 10000)
#clf = lgb.train(params, d_train, iterations)
#y_pred=clf.predict(x_test) #Create predictions on test set
params

or

In [ ]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

param_test ={'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

#n_estimators is set to a "large value". The actual number of trees build will depend on early stopping and 5000 define only the absolute maximum
clf = lgb.LGBMClassifier(max_depth=-1, random_state=314, silent=True, metric='None', n_jobs=4, n_estimators=5000)
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=n_HP_points_to_test,
    scoring='roc_auc',
    cv=3,
    refit=True,
    random_state=314,
    verbose=True)

In [ ]:
param_grid = {
        'silent': [False],
        'max_depth': [6, 10, 15, 20],
        'learning_rate': [0.001, 0.01, 0.1, 0.2, 0,3],
        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
        'gamma': [0, 0.25, 0.5, 1.0],
        'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
        'n_estimators': [100]}

In [ ]:
from sklearn.ensemble import RandomForestClassifier

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [ ]:

mlp_gs = MLPClassifier(max_iter=100)
parameter_space = {
    'hidden_layer_sizes': [(10,30,10),(20,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}
